###  Load Dataset


In [1]:
from datasets import load_dataset

split = "train"
cache_dir = "./data_cache_doc2vec_solution_notebook"

dialogue_dataset = load_dataset(
    "doc2dial",
    name="dialogue_domain",
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

document_dataset = load_dataset(
    "doc2dial",
    name="document_domain",
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

rc_dataset = load_dataset(
    "doc2dial",
    name="doc2dial_rc",
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

Reusing dataset doc2dial (./data_cache_doc2vec_solution_notebook/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)
Reusing dataset doc2dial (./data_cache_doc2vec_solution_notebook/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)
Reusing dataset doc2dial (./data_cache_doc2vec_solution_notebook/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


# Train a model for each document

### Preprocessing

In [2]:
import pandas as pd

document_full_df = pd.DataFrame(data=document_dataset)

# key=span_id, value=text
def span_dict_for_doc(doc_id):
    #use pandas as way faster
    document = document_full_df.loc[document_full_df['doc_id'] == doc_id]
    spans_dict = {}
    for span in document['spans'].iloc[0]:
        spans_dict[span['id_sp']]=span['text_sp']

    return spans_dict



In [3]:
from gensim.utils import simple_preprocess
#Getting a list of spans per grounding document !!!! Language confusion: spans are the documents in gensim land and the list of spans (aka the whole document) is the corpora in gensim

#I only want to use documents from the document datasets that have dialogues (which not all have)
unique_doc_ids = list(set(dialogue_dataset['doc_id']))

#extract all the span texts for that doc id,
#!!!Index drama. For list index starts with 0, for spans index starts with 1, use dict to keep the sp_id and doc_id
#key=doc_id, value=span dict

#TODO consider using text-sec instead of spans

def text_sec_doc(doc_id):
    #use pandas as way faster
    document = document_full_df.loc[document_full_df['doc_id'] == doc_id]
    #TODO interestingly the spans also have a text_sect which is split into less small segments and probably better
    text_sec = []
    current_text_sec = ''

    for span in document['spans'].iloc[0]:
        text = span['text_sec']
        if current_text_sec != text: #found new sect
            text_sec.append(text)
            current_text_sec = text
    return text_sec

raw_training_text_per_doc = {}
for doc_id in unique_doc_ids:
    # doc_spans[doc_id] = span_dict_for_doc(doc_id) #for spans
    raw_training_text_per_doc[doc_id] = text_sec_doc(doc_id) #for text_sec

tokenized_training_docs = {}
for doc_id in raw_training_text_per_doc:
    text_sections = raw_training_text_per_doc[doc_id]
    #simplest preprocessing from gemsim
    tokenized_sections = [list(simple_preprocess(sec, deacc=True)) for sec in text_sections]
    tokenized_training_docs[doc_id] = tokenized_sections

tokenized_training_docs.keys()

dict_keys(['Benefits Planner: Disability | Approval Process | Social Security Administration#1_0', 'Benefits Planner: Disability | How You Qualify | Social Security Administration#1_0', 'Benefits Planner | Social Security Credits and Benefit Eligibility | SSA#1_0', 'Get a driver license#1_0', 'Get a copy of a TVB ticket#1_0', 'Mustard Gas Or Lewisite Exposure | Veterans Affairs#1_0', 'Benefits Planner: Retirement | Government Pension Offset (GPO) Calculator | Social Security Administration#2_0', 'Temporary surrender of vehicle plates (store plates for a season)#3_0', 'MPN for Graduate Students | Federal Student Aid#1_0', 'Voter Registration Application Frequently Asked Questions#1_0', 'Teacher Loan Forgiveness | Federal Student Aid#1_0', 'Renew a CDL#1_0', 'Register a vehicle when you are out-of-state#1_0', 'Reconstruct Military Records Destroyed In NPRC Fire | Veterans Affairs#1_0', 'Transfer IRP registration#1_0', 'About complaints and charges for dealer and transporter businesses#3_

### training

In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

models_for_doc = {}
for doc_id in tokenized_training_docs:
    tokenized_sec = tokenized_training_docs[doc_id]
    training_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokenized_sec)]
    # Check how to fine tune the model https://radimrehurek.com/gensim/models/doc2vec.html
    model = Doc2Vec(training_docs, vector_size=10, window=4, min_count=1, workers=4, epochs=30)
    models_for_doc[doc_id] = model

len(models_for_doc)

415

### predict rc questions

In [5]:
predictions = []
references = []
for example in rc_dataset:
    question_ = example["question"]
    doc_id =example['title']
    # it does better if the user and agent string are left
    # question_ = question_.replace('user:', '')
    # question_ = question_.replace('agent:', '')

    #preprocess question in the same way
    test_doc = simple_preprocess(question_, deacc=True)
    #calculate vector using model for that document
    model = models_for_doc[doc_id]
    vector = model.infer_vector(test_doc)
    #find the most similar document (spans)
    sims = model.dv.most_similar([vector], topn=1)
    most_likely_answer = raw_training_text_per_doc[doc_id][sims[0][0]]
    # most_likely_answer = training_docs[sims[0][0] - 1]
    # most_likely_predicted_tag = most_likely_answer.tags
    #find original text for tag
    # most_likely_predicted_text = spans[most_likely_predicted_tag[0]-1]
    # print(f': {most_likely_answer.words}\n')
    # print(f'Predicted Answer: {most_likely_answer}\n')

    id_ = example["id"]
    predictions.append(
        {'id': id_,
         'prediction_text':
             most_likely_answer,
         'no_answer_probability': 0.0
         }
    )

    #just using their answers
    references.append(
        {
            "id": id_,
            "answers": example["answers"],
        }
    )

predictions[:5]


[{'id': '9f44c1539efe6f7e79b02eb1b413aa43_1',
  'prediction_text': 'A license suspension or revocation here could mean that your new home state will not issue you a license there. Remember , it is important to notify DMV of your new address so that any possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse. ',
  'no_answer_probability': 0.0},
 {'id': '9f44c1539efe6f7e79b02eb1b413aa43_3',
  'prediction_text': 'DMV maintains a point system to track dangerous drivers. Often , motorists convicted of a traffic ticket feel they have resolved all their motoring issues with the local court, but later learn that the Driver Responsibility Assessment DRA is a separate DMV charge based on the total points they accumulate. ',
  'no_answer_probability': 0.0},
 {'id': '9f44c1539efe6f7e79b02eb1b413aa43_5',
  'prediction_text': 'About ten percent of customers visiting a DMV office do not bring what they need to complete their transaction, and 

### evaluate


In [6]:
from datasets import load_metric

print(f'Number of question in train dataset {rc_dataset.shape[0]}')
metric = load_metric("squad_v2")
metric.add_batch(predictions=predictions, references=references)
final_score = metric.compute()
final_score

Number of question in train dataset 20431


{'exact': 0.7929127306543977,
 'f1': 17.246636373006577,
 'total': 20431,
 'HasAns_exact': 0.7929127306543977,
 'HasAns_f1': 17.246636373006577,
 'HasAns_total': 20431,
 'best_exact': 0.7929127306543977,
 'best_exact_thresh': 0.0,
 'best_f1': 17.246636373006577,
 'best_f1_thresh': 0.0}

### How does this compare to guessing at random?


In [7]:
import random

random_predictions = []
random_references = []
for example in rc_dataset:
    question_ = example["question"]
    doc_id =example['title']

    #pick a random text from the document
    texts = raw_training_text_per_doc[doc_id]
    most_likely_answer = texts[random.randint(0, len(texts)-1)]

    id_ = example["id"]
    random_predictions.append(
        {'id': id_,
         'prediction_text':
             most_likely_answer,
         'no_answer_probability': 0.0
         }
    )

    #just using their answers
    random_references.append(
        {
            "id": id_,
            "answers": example["answers"],
        }
    )

random_predictions[:3]

[{'id': '9f44c1539efe6f7e79b02eb1b413aa43_1',
  'prediction_text': 'We send 500,000 inquiry letters a year. If the inquiry letter does not resolve the problem , we must suspend the vehicle registration and , if it persists, your driver license!We suspend 300,000 registrations a year for failure to maintain insurance. If you fail to maintain an updated address with us , you won t learn that you have an insurance problem , and we will suspend your registration and license. Make sure you turn in your vehicle s license plates at DMV before you cancel your insurance policy. Insurance policies must be from a company licensed in New York State. ',
  'no_answer_probability': 0.0},
 {'id': '9f44c1539efe6f7e79b02eb1b413aa43_3',
  'prediction_text': 'Learn more about Insurances Lapes [2] and How to Surrender your Plates [3 ] ',
  'no_answer_probability': 0.0},
 {'id': '9f44c1539efe6f7e79b02eb1b413aa43_5',
  'prediction_text': 'About ten percent of customers visiting a DMV office do not bring what

In [8]:
print(f'Results for random text predictions for each question')
metric = load_metric("squad_v2")
metric.add_batch(predictions=random_predictions, references=random_references)
final_score = metric.compute()
final_score

Results for random text predictions for each question


{'exact': 1.5270911849640252,
 'f1': 14.51171104675806,
 'total': 20431,
 'HasAns_exact': 1.5270911849640252,
 'HasAns_f1': 14.51171104675806,
 'HasAns_total': 20431,
 'best_exact': 1.5270911849640252,
 'best_exact_thresh': 0.0,
 'best_f1': 14.51171104675806,
 'best_f1_thresh': 0.0}

### Results for validation part of rc dataset

In [9]:
rc_validation_dataset = load_dataset(
    "doc2dial",
    name="doc2dial_rc",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)



Reusing dataset doc2dial (./data_cache_doc2vec_solution_notebook/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [10]:
predictions = []
references = []
print(f'Number of question in train dataset {rc_validation_dataset.shape[0]}')

for example in rc_validation_dataset:
    question_ = example["question"]
    doc_id =example['title']
    # it does better if the user and agent string are left
    # question_ = question_.replace('user:', '')
    # question_ = question_.replace('agent:', '')

    #preprocess question in the same way
    test_doc = simple_preprocess(question_, deacc=True)
    #calculate vector using model for that document
    model = models_for_doc[doc_id]
    vector = model.infer_vector(test_doc)
    #find the most similar document (spans)
    sims = model.dv.most_similar([vector], topn=1)
    most_likely_answer = raw_training_text_per_doc[doc_id][sims[0][0]]
    # most_likely_answer = training_docs[sims[0][0] - 1]
    # most_likely_predicted_tag = most_likely_answer.tags
    #find original text for tag
    # most_likely_predicted_text = spans[most_likely_predicted_tag[0]-1]
    # print(f': {most_likely_answer.words}\n')
    # print(f'Predicted Answer: {most_likely_answer}\n')

    id_ = example["id"]
    predictions.append(
        {'id': id_,
         'prediction_text':
             most_likely_answer,
         'no_answer_probability': 0.0
         }
    )

    #just using their answers
    references.append(
        {
            "id": id_,
            "answers": example["answers"],
        }
    )

predictions[:5]

Number of question in train dataset 3972


KeyError: 'About New York State Inspections#3_0'

### Questions:
- Given that we don't have all the documents to train does that make sense for a Doc2Vec approach or does it make sense
to create a model per document?